In [3]:
# Necessary imports
import re
import emoji
import random
import pandas as pd
import numpy as np
from multiprocessing import  Pool
import time
import translators as ts

Using United States server backend.


In [5]:
# Function to remove emojis in text, since these conflict during translation
def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)


def approximate_emoji_insert(string, index,char):
    if(index<(len(string)-1)):
        
        while(string[index]!=' ' ):
            if(index+1==len(string)):
                break
            index=index+1
        return string[:index] + ' '+char + ' ' + string[index:]
    else:
        return string + ' '+char + ' ' 
    


def extract_emojis(str1):
    try:
        return [(c,i) for i,c in enumerate(str1) if c in emoji.UNICODE_EMOJI]
    except AttributeError:
        return []

In [6]:
import os
if not os.path.exists('translatedtrain'):
    os.makedirs('translatedtrain')
if not os.path.exists('translatedval'):
    os.makedirs('translatedval')
if not os.path.exists('translatedtest'):
    os.makedirs('translatedtest')

In [7]:
# Use multiprocessing framework for speeding up translation process
def parallelize_dataframe(df, func, n_cores=1):
    '''parallelize the dataframe'''
    # df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    # df = pd.concat(pool.map(func, df))
    # pool.close()
    # pool.join()
    # print("done parallelizing")
    return func(df)

# Main function for translation
def translate(x,lang):
    '''provide the translation given text and the language'''
    #x=preprocess_lib.preprocess_multi(x,lang,multiple_sentences=False,stop_word_remove=False, tokenize_word=False, tokenize_sentence=False)
    # translator = Translator()
    # translator = Translator(to_lang='en',from_lang='pt')
    emoji_list=extract_emojis(x)
    # print("try")
    # translated_text = translator.translate(x, src = 'pt', dest='en')
    # translated_text = translated_text.text
    try:
        # print("try")
        # translated_text = translator.translate(x)
        # translated_text=translate_text(x,'pt','en')
        translated_text= ts.google(x, from_language='id', to_language='en')
        # print("translated successfully")
    except:
        translated_text=x
        print("not translated")
    for ele in emoji_list:
        translated_text=approximate_emoji_insert(translated_text, ele[1],ele[0])
    # print("done translating")
    return translated_text

def add_features(df):
    '''adding new features to the dataframe'''
    translated_text=[]
    for index,row in df.iterrows():
        # if('pt' in ['en','unk']):
        #     translated_text.append(row['text'])
        # else:
        translated_text.append(translate(row['text'], 'id'))    
    df["translated"]=translated_text
    return df

In [21]:
import glob 
train_files = glob.glob('/Dataset/train/*csv')
test_files = glob.glob('test/*.csv')
val_files = glob.glob('val/*.csv')
files= train_files+test_files+val_files
print(files)

[]


In [14]:
from tqdm import tqdm_notebook
size=10

for file in files:
    wp_data=pd.read_csv(file)
    end= len(wp_data.index)
    list_df=[]
    for i in tqdm_notebook(range(0,end,size)):
                print(i,"_iteration")
                df_new=parallelize_dataframe(wp_data[i:i+size],add_features,n_cores=1)
                list_df.append(df_new)
    df_translated=pd.concat(list_df,ignore_index=True)
    file_name='translated'+file
    df_translated.to_csv(file_name,index=False)

In [15]:
import translators as ts

In [16]:
ts.google(word, from_language='id')

NameError: name 'word' is not defined